 # Developing a system for categorizing movie reviews 

'Junky Union', a community for classic movie lovers, is developing a system for filtering and categorizing movie reviews. The goal is to train a model to detect negative reviews automatically.

## Contents <a id='back'></a>

* [Intro](#intro)
* [Stage 1. Data loading and preprocessing](#data_review)
    * [1.1 Data exploration](#explore_data)
* [Stage 2. Data division](#data_division)
* [Stage 3. Model analysis](#model_analysis)
    * [3.1 Decision tree](#decision_tree)
    * [3.2 Random forest](#random_forest)
    * [3.3 Logistic regression](#logistic_regression)
    * [3.4 Model comparison](#model_comparison)
* [Stage 4. Model quality](#model_quality)
    * [4.1 Decision tree](#decision_tree_quality)
    * [4.2 Random forest](#random_forest_quality)
    * [4.3 Logistic regression](#logistic_regression_quality)
* [Stage 5. Sanity test](#sanity_test)
* [Conclusion](#conclusion)

## Introduction <a id='intro'></a>

We will use a dataset of IMDb (Internet Movie Database) with polarity legends to build a model to classify positive and negative reviews. This must reach an F1 value of at least 0.85.

## Stage 1. Data loading and preprocessing <a id='data_review'></a>

In [1]:
# Loading all libraries
import pandas as pd

In [2]:
# Load the data files into DataFrame
df = pd.read_csv('https://code.s3.yandex.net/datasets/imdb_reviews.tsv', sep='\t')

In [4]:
df.head()

,tconst,title_type,primary_title,original_title,start_year,end_year,runtime_minutes,is_adult,genres,average_rating,votes,review,rating,sp,pos,ds_part,idx
0,tt0068152,movie,$,$,1971,\N,121,0,"Comedy,Crime,Drama",6.3,2218.0,The pakage implies that Warren Beatty and Gold...,1,neg,0,train,8335
1,tt0068152,movie,$,$,1971,\N,121,0,"Comedy,Crime,Drama",6.3,2218.0,How the hell did they get this made?! Presenti...,1,neg,0,train,8336
2,tt0313150,short,'15','15',2002,\N,25,0,"Comedy,Drama,Short",6.3,184.0,There is no real story the film seems more lik...,3,neg,0,test,2489
3,tt0313150,short,'15','15',2002,\N,25,0,"Comedy,Drama,Short",6.3,184.0,Um .... a serious film about troubled teens in...,7,pos,1,test,9280
4,tt0313150,short,'15','15',2002,\N,25,0,"Comedy,Drama,Short",6.3,184.0,I'm totally agree with GarryJohal from Singapo...,9,pos,1,test,9281
